# Install and Import Dependencies

In [1]:
%pip install opencv-python
%pip install mediapipe
%pip install tensorflow==2.4.1
%pip install tensorflow-gpu==2.4.1
%pip install sklearn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==2.4.1


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt

# Extracting and Drawing Joints
<img src="https://www.researchgate.net/profile/Jungpil-Shin/publication/362351225/figure/fig3/AS:1183521695645696@1659184969422/Mediapipe-detects-33-nodes-of-the-human-pose.ppm" width="500px">

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

In [8]:
cap = cv2.VideoCapture(0)
with  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened(): 
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        # Render detections
        draw_landmarks(image, results)

        cv2.imshow('Human GAIT Identification', image)
        
        if cv2.waitKey(10) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

In [18]:
#number of detected landmarks
len(results.pose_landmarks.landmark)

33

In [31]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

In [32]:
result_test = extract_keypoints(results)
result_test

array([ 5.33204436e-01,  3.53722215e-01, -1.11866856e+00,  9.97585118e-01,
        5.60662031e-01,  2.85479695e-01, -1.06956565e+00,  9.95640159e-01,
        5.79667449e-01,  2.84901947e-01, -1.06966901e+00,  9.95842636e-01,
        5.97794712e-01,  2.86072046e-01, -1.06995273e+00,  9.94155884e-01,
        5.12273729e-01,  2.94160455e-01, -1.06048727e+00,  9.97253954e-01,
        4.98674065e-01,  2.98513353e-01, -1.06037521e+00,  9.97979164e-01,
        4.86103833e-01,  3.03281277e-01, -1.06096566e+00,  9.97840643e-01,
        6.29389226e-01,  3.07845503e-01, -6.47804141e-01,  9.96171713e-01,
        4.75476682e-01,  3.26999396e-01, -5.97448349e-01,  9.98833299e-01,
        5.74326277e-01,  4.07171130e-01, -9.50631917e-01,  9.98288751e-01,
        5.11687100e-01,  4.13326412e-01, -9.36698556e-01,  9.99069810e-01,
        7.84441590e-01,  5.88353157e-01, -2.36885637e-01,  9.93393123e-01,
        3.58175069e-01,  5.72574198e-01, -3.48331034e-01,  9.98146474e-01,
        9.71885860e-01,  

# 4. Setup Folders for Collection

In [21]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Custom_Activity_Dataset') 

# Actions that we try to detect
actions = np.array(['lance','samer','seif'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [34]:
for action in actions:
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Importing Casia-B Dataset